In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import RandomizedSearchCV
import pickle
import json

from sklearn.metrics import r2_score

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Data cleaning and wrangling for testing data according to training data

In [30]:

df = pd.read_csv('./data/train.csv',index_col=['Id'])
train = df.drop('SalePrice',axis=1)
test = pd.read_csv('./data/test.csv',index_col=['Id'])
train.shape,test.shape

((1460, 79), (1459, 79))

In [31]:
# re-arrange data frame
CatCols = [col for col in train.columns if train[col].dtype == 'object']
CatCols.append('MSSubClass')
NumCols = [col for col in train.columns if col not in CatCols]
train['MSSubClass']=train['MSSubClass'].astype('str')
test['MSSubClass']=test['MSSubClass'].astype('str')

In [32]:
# Fill NA with 0
for col in NumCols:
    train[col] = train[col].fillna(0)
    test[col] = test[col].fillna(0)
for col in CatCols:
    train[col] = train[col].fillna('0')
    test[col] = test[col].fillna('0')

In [33]:
# re-organize columns
train_cat = train[CatCols].astype('category')
train_num = train[NumCols]
train = train_cat.join(train_num,on='Id')

test_cat = test[CatCols].astype('category')
test_num = test[NumCols]
test = test_cat.join(test_num,on='Id')


In [34]:
# data wrangling
#change the year to age

for col in ['YearBuilt','YearRemodAdd','GarageYrBlt']:
    train[col] = train['YrSold'] - train[col]
    test[col] = test['YrSold'] - test[col]
    
maxage = train.GarageYrBlt.max() + 10
train.GarageYrBlt = train['GarageYrBlt'].replace(0, maxage)
test.GarageYrBlt = test['GarageYrBlt'].replace(0,maxage)

## transform categorical data to analyzable numbers

In [8]:
cat_mask = (train.dtypes == 'category')


In [9]:
col_map = json.load(open('column_map.json'))
le_map = json.load(open('le_map.json'))

In [35]:
# Use LabelEncoder to preprocess Categorical data from string to integer

le = LabelEncoder() 

train_le = train.copy()
test_le = test.copy()
abnormal_columns=[]
for col in CatCols:
    try:
        train_le[col] = le.fit_transform(train_le[col])
        test_le[col] = le.transform(test_le[col])
    except:
        abnormal_columns.append(col)

In [12]:
# Those abnormal columns contain un-seen category in test data but not in training data
abnormal_columns

['MSZoning',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'KitchenQual',
 'Functional',
 'SaleType',
 'MSSubClass']

In [36]:
# Those abnormal un-seen category are mostly NA data.
ab_var_dic = {}
for col in abnormal_columns:

    ab_vars = [var for var in test[col].unique() if var not in le_map[col]]
    ab_var_dic[col] = ab_vars

ab_var_dic

{'MSZoning': ['0'],
 'Utilities': ['0'],
 'Exterior1st': ['0'],
 'Exterior2nd': ['0'],
 'KitchenQual': ['0'],
 'Functional': ['0'],
 'SaleType': ['0'],
 'MSSubClass': ['150']}

## deal with NA

### First way to fill NA is assign a reasonble value, for example, fill with neighborhood's most popular value

In [39]:
# Deal with 'MSZoning'
test.loc[test['MSZoning'] == '0'][['MSZoning','Neighborhood']]

,MSZoning,Neighborhood
Id,,
1916,0,IDOTRR
2217,0,IDOTRR
2251,0,IDOTRR
2905,0,Mitchel


In [42]:
test.loc[test['Neighborhood'] == 'IDOTRR'][['MSZoning','Neighborhood']].groupby('MSZoning').count()


,Neighborhood
MSZoning,
0,3
C (all),13
RM,40


In [44]:
test['MSZoning'] = test['MSZoning'].replace('0','RM')

In [47]:
# Deal with Utilities
test.loc[test['Utilities'] == '0'][['Utilities','Neighborhood']]

,Utilities,Neighborhood
Id,,
1916,0,IDOTRR
1946,0,Gilbert


In [51]:
test.loc[test['Neighborhood'] == 'IDOTRR'][['Utilities','Neighborhood']].groupby('Utilities').count()

,Neighborhood
Utilities,
0,1
AllPub,55


In [52]:
test['Utilities'] = test['Utilities'].replace('0','AllPub')

### This might not work for some features the neighborhood value is quite diverse. Then we just fill with most common value in whole database

In [42]:
for col in abnormal_columns:
    mode = test[col].mode()
    print(ab_var_dic[col][0],mode[0])
    test[col] = test[col].replace(ab_var_dic[col][0],mode[0])

0 RL
0 AllPub
0 VinylSd
0 VinylSd
0 TA
0 Typ
0 WD
150 20


In [43]:
# Those abnormal un-seen category are mostly NA data.
for col in abnormal_columns:
    print(col)
    ab_vars2 = [var for var in test[col].unique() if var not in le_map[col]]

    print(ab_vars2)

MSZoning
[]
Utilities
[]
Exterior1st
[]
Exterior2nd
[]
KitchenQual
[]
Functional
[]
SaleType
[]
MSSubClass
[]


In [54]:
test.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1461,RH,Pave,0,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,...,730.0,140,0,0,0,120,0,0,6,2010
1462,RL,Pave,0,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,...,312.0,393,36,0,0,0,0,12500,6,2010
1463,RL,Pave,0,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,482.0,212,34,0,0,0,0,0,3,2010
1464,RL,Pave,0,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,470.0,360,36,0,0,0,0,0,6,2010
1465,RL,Pave,0,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,...,506.0,0,82,0,0,144,0,0,1,2010


In [52]:
# Label Encoder
for col in abnormal_columns:

    test_le[col] = test[col].map(le_map[col])

In [56]:
test_le.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1461,2,1,0,3,3,0,4,0,12,1,...,730.0,140,0,0,0,120,0,0,6,2010
1462,3,1,0,0,3,0,0,0,12,2,...,312.0,393,36,0,0,0,0,12500,6,2010
1463,3,1,0,0,3,0,4,0,8,2,...,482.0,212,34,0,0,0,0,0,3,2010
1464,3,1,0,0,3,0,4,0,8,2,...,470.0,360,36,0,0,0,0,0,6,2010
1465,3,1,0,0,1,0,4,0,22,2,...,506.0,0,82,0,0,144,0,0,1,2010


In [53]:
test_le.to_csv('./data/test_le.csv')
train_le.to_csv('./data/train_le.csv')

In [57]:
# OneHotEncoder
ohe = OneHotEncoder(categorical_features = cat_mask, sparse=False)
train_ohe = ohe.fit_transform(train_le)
test_ohe = ohe.transform(test_le)

//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [59]:
ohe_column_names = ohe.get_feature_names()
column_names = np.concatenate((ohe_column_names,NumCols))
len(column_names)

318

In [60]:
pd.DataFrame(test_ohe, columns=column_names).to_csv('./data/test_ohe.csv')
pd.DataFrame(train_ohe,columns=column_names).to_csv('./data/train_ohe.csv')

In [61]:
# Scaling
scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
train_scale = scaler.fit_transform(train_ohe)
test_scale = scaler.transform(test_ohe)

In [62]:
pd.DataFrame(test_scale, columns=column_names).to_csv('./data/test_scale.csv')
pd.DataFrame(train_scale,columns=column_names).to_csv('./data/train_scale.csv')